In [1]:
import pandas as pd
from Testmodule import 資產負債表, 綜合損益表, 證交所產業分類
from datetime import date

# 年度
current_date = date.today()
user_year = current_date.year

# 上一季資料
current_quarter = (current_date.month - 1) // 3 + 1
user_quarter = current_quarter - 1 
if user_quarter == 1:
    user_quarter = 4
    user_year -= 1  #如果是第一季，user_year改為上個年度

income_statement = 綜合損益表.綜合損益表by季(user_year, user_quarter)
balance_sheet = 資產負債表.資產負債表by季(user_year, user_quarter)

#兩張table都有獨立的key，利用這兩個key值將兩張table結合
resultdf = income_statement.merge(balance_sheet,on='Key',how='inner')
resultdf = resultdf.drop(columns=['公司 代號_y', '年度_y', '季別_y', '公司名稱_y']) #刪掉重複欄位
resultdf.rename(columns = {'公司 代號_x':'公司代號', '年度_x':'年度', '季別_x':'季別', '公司名稱_x':'公司名稱' }, inplace = True) #重新命名欄位
resultdf['公司代號'] = resultdf['公司代號'].astype(str) #變更欄位的type，用於merge用
category = 證交所產業分類.industry()
resultdf = resultdf.merge(category,how = "inner",on = "公司代號")
resultdf


,公司代號,年度,季別,公司名稱,營業收入,營業成本,原始認列生物資產及農產品之利益（損失）,生物資產當期公允價值減出售成本之變動利益（損失）,營業毛利（毛損）,未實現銷貨（損）益,...,歸屬於母公司業主之權益合計,共同控制下前手權益,合併前非屬共同控制股權,非控制權益,權益總計,待註銷股本股數（單位：股）,預收股款（權益項下）之約當發行股數（單位：股）,母公司暨子公司所持有之母公司庫藏股股數（單位：股）,每股參考淨值,產業別
0,1101,112,3,台泥,80966464,66737333,--,--,14229131,--,...,222398369,--,--,22328122,244726491,--,0,20000000,30.32,水泥工業
1,1101,112,2,台泥,53964171,45509992,--,--,8454179,--,...,214284220,--,--,20435231,234719451,--,0,20000000,29.21,水泥工業
2,1101,112,1,台泥,26295929,23826708,--,--,2469221,--,...,220077058,--,--,20003976,240081034,--,0,24000000,30.02,水泥工業
3,1101,111,4,台泥,113929706,104345739,--,--,9583967,--,...,217894766,--,--,20381014,238275780,--,0,4000000,29.64,水泥工業
4,1101,111,3,台泥,79274330,74872991,--,--,4401339,--,...,201413996,--,--,20256450,221670446,--,0,4000000,29.05,水泥工業
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4767,9958,112,3,世紀鋼,9539392,8275189,--,--,1264203,--,...,7895925,--,--,3434276,11330201,--,0,0,33.59,鋼鐵工業
4768,9958,112,2,世紀鋼,5894102,5299030,--,--,595072,--,...,7354916,--,--,3214123,10569039,--,0,0,31.53,鋼鐵工業
4769,9958,112,1,世紀鋼,2675284,2510328,--,--,164956,--,...,6966880,--,--,3226055,10192935,--,0,0,29.98,鋼鐵工業
4770,9958,111,4,世紀鋼,9222058,8779554,--,--,442504,--,...,6959252,--,--,3281807,10241059,--,0,0,29.95,鋼鐵工業


In [4]:
resultdf.iloc[0]

公司代號                              1101
年度                                 112
季別                                   3
公司名稱                                台泥
營業收入                          80966464
營業成本                          66737333
原始認列生物資產及農產品之利益（損失）                 --
生物資產當期公允價值減出售成本之變動利益（損失）            --
營業毛利（毛損）                      14229131
未實現銷貨（損）益                           --
已實現銷貨（損）益                           --
營業毛利（毛損）淨額                    14229131
營業費用                           7632475
其他收益及費損淨額                           --
營業利益（損失）                       6596656
營業外收入及支出                       3943167
稅前淨利（淨損）                      10539823
所得稅費用（利益）                      3035124
繼續營業單位本期淨利（淨損）                 7504699
停業單位損益                              --
合併前非屬共同控制股權損益                       --
本期淨利（淨損）                       7504699
其他綜合損益（淨額）                     2925335
合併前非屬共同控制股權綜合損益淨額                   --
本期綜合損益總額                      10430034
淨利（淨損）歸屬於母公司業主           

In [5]:
# dataset1
# 把resultdf另外用project這個dictionary給複製出來，避免改動到原始爬蟲資料
project = {}
project['公司代號'] = resultdf['公司代號']
project['公司名稱'] = resultdf['公司名稱']
project['產業別'] = resultdf['產業別']
project['年度'] = resultdf['年度'] 
project['季別'] = resultdf['季別']
project['基本每股盈餘_EPS'] = resultdf['基本每股盈餘（元）'] #EPS(1)
project['營業收入'] = resultdf['營業收入']
project['本期淨利（淨損）'] = resultdf['本期淨利（淨損）'] #利潤(6)

project = pd.DataFrame(project)
project

,公司代號,公司名稱,產業別,年度,季別,基本每股盈餘_EPS,營業收入,本期淨利（淨損）
0,1101,台泥,水泥工業,112,3,0.82,80966464,7504699
1,1101,台泥,水泥工業,112,2,0.64,53964171,5212327
2,1101,台泥,水泥工業,112,1,0.20,26295929,1005644
3,1101,台泥,水泥工業,111,4,0.74,113929706,4157085
4,1101,台泥,水泥工業,111,3,0.38,79274330,1554383
...,...,...,...,...,...,...,...,...
4767,9958,世紀鋼,鋼鐵工業,112,3,2.52,9539392,643596
4768,9958,世紀鋼,鋼鐵工業,112,2,1.20,5894102,209438
4769,9958,世紀鋼,鋼鐵工業,112,1,0.20,2675284,-7168
4770,9958,世紀鋼,鋼鐵工業,111,4,1.59,9222058,100962


In [6]:
# dataset1
import pandas as pd
from pandas.core.reshape.pivot import pivot
# 先將project的年度、季別合併，變成date的形式後排序，之後繪圖座標軸才會正確
project['Date'] = (project['年度'] + 1911).astype(str)+ (project['季別']*3-2).astype(str)
project['Date'] = pd.to_datetime(project['Date'], format='%Y%m')

import datetime
p_time = project #複製一個要修改的p_time避免動到原本project
p_time = p_time.drop_duplicates(subset=['Date']) #把重複的時間刪掉，因此只會有5個時間從最新(當季)到最舊(前5季)
time_list = []
for i in range(5):
    time_list.append(p_time.iat[i,-1].strftime("%Y-%m-%d")) #用for迴圈把每個時間加到time_list
time_list = sorted(time_list, reverse=True) #由最新的季別排到最早的季別

# 處理missing
project1 = project.dropna(subset=['基本每股盈餘_EPS','營業收入', '本期淨利（淨損）']) #n=4772
project1 = project1.groupby(['公司名稱']).agg('count') #n=958
project1 = project1[~(project1['基本每股盈餘_EPS'] < 5)] #n=943
project1 = pd.merge(project1, project, how = 'left', suffixes=["_x",""], on = ['公司名稱']) #n=4715
project1 = project1[['產業別','公司代號','公司名稱','基本每股盈餘_EPS','營業收入', '本期淨利（淨損）','Date','季別','年度']] #n=4715
project1

# exclude1: 2023/Q3>2022/Q3 營收
p1 = project1[['公司名稱','Date','營業收入']]
p1.sort_values(by = ['公司名稱','Date'],) #排序資料
p1 = p1.pivot(index = '公司名稱',columns = 'Date',values = '營業收入') #轉置表格 #n=943
p1 = p1[~(p1[time_list[0]] < p1[time_list[4]])] #n=319
p1 = pd.merge(p1, project1, how = 'left', on = ['公司名稱']) #n=1595
p1 = p1[['產業別','公司代號','公司名稱','基本每股盈餘_EPS','營業收入', '本期淨利（淨損）','Date']] #n=1595

# exclude2 : 任一季EPS為負數
p2 = p1[['公司名稱','Date','基本每股盈餘_EPS']]
p2.sort_values(by = ['公司名稱','Date'])
p2 = p2.pivot(index = '公司名稱',columns = 'Date',values = '基本每股盈餘_EPS') #n=319
for time_ymd in time_list:
    p2 = p2[~(p2[time_ymd] < 0)] #n=233 用迴圈跑time_list取代固定季別，之後可以擴充更多季別
p2 = pd.merge(p2, project, how = 'left', on = ['公司名稱'])
p2 = p2[['產業別','公司代號','公司名稱','基本每股盈餘_EPS','營業收入', '本期淨利（淨損）','Date']] #n=1165

# exclude3: 2023/Q3>2022/Q3 利潤
p3 = p2[['公司名稱','Date','本期淨利（淨損）']]
p3.sort_values(by = ['公司名稱','Date']) #排序資料
p3 = p3.pivot(index = '公司名稱',columns = 'Date',values = '本期淨利（淨損）') #轉置表格 #n=233
p3 = p3[~(p3[time_list[0]] < p3[time_list[4]])] #n=167
p3 = pd.merge(p3, project1, how = 'left', on = ['公司名稱'])
p3 = p3[['產業別','公司代號','公司名稱','基本每股盈餘_EPS','營業收入', '本期淨利（淨損）','Date']] #n=4710

#篩選出年收成長前5%支
dataset1 = p3.pivot(index = '公司名稱',columns = 'Date',values = '營業收入')
dataset1['營收成長率(%)'] = ((dataset1[time_list[0]]-dataset1[time_list[4]])/dataset1[time_list[0]])*100
dataset1.sort_values(by = ['營收成長率(%)','公司名稱'],ascending=False) #排序資料
dataset1=dataset1.head(int((len(dataset1.axes[0])-1)*0.05)) #n=8
dataset1

Date,2022-07-01 00:00:00,2022-10-01 00:00:00,2023-01-01 00:00:00,2023-04-01 00:00:00,2023-07-01 00:00:00,營收成長率(%)
公司名稱,,,,,,
jpp-KY,1270244,1789193,559071,1121128,1765747,28.061948
三商電,2661769,3788642,919475,1689773,2678307,0.617480
三能-KY,1368722,1861962,397224,915520,1403625,2.486633
三陽工業,38132750,50850618,15334065,32721717,49408243,22.821077
世紀鋼,6277421,9222058,2675284,5894102,9539392,34.194747
世芯-KY,9143073,13725204,5716430,13644904,21254900,56.983693
中保科,11336609,15357445,4067376,8337552,12624832,10.203882
中磊,46166568,64573720,15659134,31325533,46950847,1.670426


In [2]:
# dataset1
import pandas as pd
import matplotlib.pyplot as plt

# 先將project的年度、季別合併，變成date的形式後排序，之後繪圖座標軸才會正確
project['Date'] = (project['年度'] + 1911).astype(str)+ (project['季別']*3-2).astype(str)
project['Date'] = pd.to_datetime(project['Date'], format='%Y%m')

project = project.sort_values(by='Date')
project = pd.DataFrame(project)


def plot_company_data(company_code):

    company_data = project[project['公司代號'] == company_code]
    plt.figure(figsize=(10, 6))

    # Define a custom date formatter
    date_format = "%YQ%q"
    plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter(date_format))

    # Plot the data
    plt.plot(company_data['Date'], company_data['營業收入'], label='營業收入', marker='o')
    plt.title('Revenue')
    
    # 設定 x 軸刻度為 Quarter，並使用自定義的日期格式
    quarters = company_data['Date'].dt.to_period("Q")
    quarters_str = quarters.astype(str)
    plt.xticks(company_data['Date'].unique(), quarters_str.unique())
    
    # 設定 y 軸刻度平均分成 5 等份
    min_value = company_data['營業收入'].min()
    max_value = company_data['營業收入'].max()
    tick_interval = (max_value - min_value) / 5
    plt.yticks([min_value + i * tick_interval for i in range(6)])
    
    plt.xlabel('Quarter')
    plt.ylabel('Amount')
    plt.legend()
    plt.grid(True)
    plt.show()

company_code = int(input('輸入公司代號'))
plot_company_data(company_code)
# test

NameError: name 'project' is not defined

In [3]:
# dataset2，使用者自行選擇產業，之後依ROE排序列印出來
dataset2 = resultdf
# industry_list = [str(i)for i in list(set(證交所產業分類.industry()['產業別'])) if i]
industry_list = [str(i)for i in list(set(dataset2['產業別'])) if i]

In [5]:
# 把產業別印出來(總不能不告訴使用者有哪些產業吧?)
print(industry_list)

# 先篩選出產業，直到輸入正確的產業
while True:
    try:
        industry_input = str(input('請輸入產業：'))
        if industry_input in industry_list:
            break
    except:
        print('請輸入正確的產業名稱')
        continue

# 先按公司、年度、季別排序 
dataset2 = dataset2.sort_values(by=['公司代號', '年度', '季別'])

# 將季別轉換為數值
dataset2['季別'] = pd.to_numeric(dataset2['季別'])

# 計算當季與前一季權益總計的平均
dataset2['權益_avg'] = (dataset2['權益總計'] + dataset2.groupby('公司代號')['權益總計'].shift(1)) / 2

# 計算當季淨利
dataset2['本期淨利_diff'] = dataset2['本期淨利（淨損）']
# 如果不是第一季，計算與前一季差異
mark = dataset2['季別'] != 1
dataset2.loc[mark, '本期淨利_diff'] = dataset2.groupby('公司代號')['本期淨利（淨損）'].diff()


filtered_data = dataset2[dataset2['產業別'] == industry_input]
selected_data = filtered_data[['公司代號', '年度', '季別', '公司名稱']]#, '營業收入', '本期淨利（淨損）', '每股參考淨值', '本期綜合損益總額']]
selected_data['ROE'] = filtered_data['本期淨利_diff'] / filtered_data['權益_avg']

# 使用 pivot_table 進行轉換
pivot_df = selected_data.pivot_table(index=['公司代號', '公司名稱'], columns=['年度', '季別'], values='ROE').reset_index()
pivot_df

# 建立每個公司的平均 ROE
# filtered_data['過去五季平均ROE'] = filtered_data['本期綜合損益總額'] / filtered_data['權益總計']
# AvgROE = filtered_data.groupby(['公司代號','公司名稱'])['過去五季平均ROE'].mean().reset_index()

# # 按大小排序
# Dataset2 = AvgROE.sort_values(by='過去五季平均ROE', ascending=False)

['綠能環保', '建材營造業', '觀光餐旅', '橡膠工業', '資訊服務業', '電子零組件業', '通信網路業', '電腦及週邊設備業', '運動休閒', '玻璃陶瓷', '塑膠工業', '其他業', '汽車工業', '數位雲端', '油電燃氣業', '食品工業', '化學工業', '貿易百貨業', '半導體業', '紡織纖維', '其他電子業', '鋼鐵工業', '造紙工業', '電子通路業', '電器電纜', '電機機械', '水泥工業', '航運業', '光電業', '居家生活', '生技醫療業']


C:\Users\e1993\AppData\Local\Temp\ipykernel_25472\4232282304.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_data['ROE'] = filtered_data['本期淨利_diff'] / filtered_data['權益_avg']


年度  公司代號    公司名稱       111       112                    
季別                       4         1         2         3
0   2208      台船 -0.111930 -0.190857 -0.125769 -0.139788
1   2603      長榮  0.054431  0.010848  0.012465  0.049161
2   2605      新興  0.016255  0.007210  0.009048  0.005964
3   2606      裕民  0.014943  0.014559  0.017878  0.022484
4   2607      榮運  0.019128  0.015432  0.040483  0.024651
5   2608    嘉里大榮  0.029396  0.023986  0.070808  0.025024
6   2609      陽明  0.043864  0.010028 -0.000042  0.010268
7   2610      華航 -0.012392  0.019059  0.046569  0.033404
8   2611      志信  0.005394  0.002305  0.024237  0.064544
9   2612      中航  0.017108  0.014944  0.004309  0.005425
10  2613      中櫃  0.007891 -0.009870  0.012990  0.012347
11  2615      萬海 -0.000189 -0.009424 -0.010818  0.011880
12  2617      台航  0.025555  0.009063  0.010779  0.070819
13  2618     長榮航  0.009585  0.048822  0.063879  0.060205
14  2630      亞航  0.002448  0.001368 -0.014949  0.019088
15  2633    台灣高鐵  0.024456  0.031112  0.023813  0.018317
16  2634      漢翔  0.016177  0.033761  0.046325  0.039921
17  2636    台驊投控  0.014255  0.030045  0.061853  0.064417
18  2637   慧洋-KY  0.022441 -0.002853  0.033387  0.009445
19  2642     宅配通  0.017166  0.000660  0.018132  0.013877
20  2645    長榮航太  0.013625  0.026729  0.053638  0.050251
21  5607     遠雄港  0.039450  0.028271  0.024427  0.045818
22  5608     四維航  0.020273 -0.020984 -0.028489 -0.018521
23  6753    龍德造船  0.067295  0.066872  0.030574  0.035674
24  6757  台灣虎航-創 -0.347009  0.371205  0.167836  0.181601
25  8367    建新國際  0.018221  0.024647  0.014027  0.013855